In [3]:
#pip install pandas

In [50]:
import pandas as pd
import re

#le

#read in spreadsheet of all members from https://github.com/unitedstates/congress-legislators
all_data = pd.read_csv("legislators-current.csv")


#read in the partisan lean for states and districts from https://github.com/fivethirtyeight/data/tree/master/partisan-lean
state_lean = pd.read_csv("fivethirtyeight_partisan_lean_STATES.csv")

district_lean = pd.read_csv("fivethirtyeight_partisan_lean_DISTRICTS.csv")

#list of the states to filter out non-voting members from PR and AS 
states = list(state_lean["abbreviation"].values)

#read in the twitter handles we collectedly manually
twitter_handles = pd.read_csv("twitter_handles.csv")


In [51]:
columns = list(all_data.columns.values).

#re-order columns for ease of reading

new_columns = ['full_name','gender', 'type', 'state', 'district', 'party', 'official_twitter', 'last_name', 'first_name', 'middle_name', 'suffix','url', 'address', 'phone', 'contact_form', 'rss_url', 'facebook', 'youtube', 'youtube_id', 'bioguide_id', 'thomas_id', 'opensecrets_id', 'lis_id', 'fec_ids', 'cspan_id', 'govtrack_id', 'votesmart_id', 'ballotpedia_id', 'washington_post_id', 'icpsr_id', 'wikipedia_id']
all_data = all_data[new_columns]

In [52]:
#create a column for districts that has state and number
all_data.insert(5, "full_district", 'NaN')

In [53]:
# get the district labels to match the partisan lean data and change party names
for index_label, row_series in all_data.iterrows():
    #change party name
    if all_data.at[index_label, 'party'] == 'Republican':
        all_data.at[index_label, 'party'] = "R"
    if all_data.at[index_label, 'party'] == 'Democrat':
        all_data.at[index_label, 'party'] = "D"
    
    # we only do this part for the house
    if all_data.at[index_label, 'type'] == 'rep':
        # the spreadhsheet has at-large districts like alaska as 0th, but the 538 data has them as one so fix that
        if row_series['district'] == 0:
            all_data.at[index_label , 'full_district'] = row_series['state'] + "-" + str(1)
        # For each row update the full_district
        else:
            all_data.at[index_label , 'full_district'] = row_series['state'] + "-" + str(row_series['district'])[:-2]


In [54]:
#insert the columns for partisan lean for 2018 [9] and 2020 [10] and personal twitter [7]

all_data.insert(7, "personal_twitter", 'NaN')
all_data.insert(9, "partisan_lean_2018", 'NaN')
all_data.insert(10, "partisan_lean_2020", 'NaN')
#all_data.head()

In [56]:
for index_label, row_series in all_data.iterrows():
    #senators use state lean
    if all_data.at[index_label, 'type'] == 'sen':
        #2018 partisan lean
        #get the current state abbreviation
        current_state = row_series['state']
        #get the lean of that state 
        state_lean_2018 = state_lean.loc[state_lean['abbreviation'] == current_state, '2018'].iloc[0]
        all_data.at[index_label , 'partisan_lean_2018'] = state_lean_2018
        #2020 partisan lean
        state_lean_2020 = state_lean.loc[state_lean['abbreviation'] == current_state, '2020'].iloc[0]
        all_data.at[index_label , 'partisan_lean_2020'] = state_lean_2020
        
    #house uses district lean
    if all_data.at[index_label, 'type'] == 'rep':
        #get the district and state
        current_dist = row_series['full_district']
        current_state = row_series['state']
        #print(current_state)
        if current_state in states:
            #print(current_dist)
            #get the lean
            dist_lean_2018 = district_lean.loc[district_lean['district'] == current_dist, '2018'].iloc[0]
            #add it
            all_data.at[index_label , 'partisan_lean_2018'] = dist_lean_2018
            #rinse and repeat for 2020
            dist_lean_2020 = district_lean.loc[district_lean['district'] == current_dist, '2020'].iloc[0]
            all_data.at[index_label , 'partisan_lean_2020'] = dist_lean_2020


In [69]:
for index_label, row_series in all_data.iterrows():
    #use the bioguide_id to match (bioguide id is a number from govtrack that lets us match unique members)
    id_code = row_series['bioguide_id']
    state = row_series['state']
    #get the lean of that state 
    twitter_url = twitter_handles.loc[twitter_handles['bioguide_id'] == id_code, 'twitter_url'].iloc[0]
    #try to catch for NaN
    try:
        personal_twitter = re.sub(r'https:\/\/twitter.com\/', "", twitter_url,)
    except:
        #print("no twitter!")
        personal_twitter = "NaN"

    all_data.at[index_label , 'personal_twitter'] = personal_twitter
        
   

In [1]:
#write data to project data
all_data.to_csv('project_data.csv', index = False)


NameError: name 'all_data' is not defined